In [1]:
#模型的保存与加载
'''
介绍
本实验主要讲解了在不同环境下，如何使用 PyTorch 对模型进行加载的过程。
在本实验中我们将学到 torch.save() 、 torch.load() 和 torch.nn.Module().loadstatedict() 的作用以及使用方式。

知识点
完整模型的保存
模型参数的保存
模型的加载
'''

'''
模型的保存与加载
模型训练的实质就是优化模型中的参数，使模型损失最小的过程。而模型保存其实也有两种方式，
一种是直接保存整个模型，另一种就是保存模型的参数。接下来，让我们以一个简单的模型为例子。

下面我们建立了一个简单的全连接网络模型：
'''

import torch
import torch.nn as nn


class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred


# 输入层为6个神经元节点
model = Model(n_input_features=6)
model

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [2]:
'''
整个模型的保存与加载
我们可以将整个模型直接进行保存，使用 torch.save(model, FILE) 即可，其中 model 为模型的变量名，FILE 为想要保存的文件路径。
'''
# 将文件保存为 model.pth
FILE = "./model/tmpmodel.pth"
torch.save(model, FILE)
print("保存成功")

保存成功


In [3]:
#接下来让我们使用 torch.load(FILE) 来对模型进行加载：

# 由于模型中已经有了结构和参数，因此我们可以直接用一个新的变量接它即可
loaded_model = torch.load(FILE)
# 再展示之前，必须需要告诉模型现在在做模型评估，避免模型自动梯度下降
loaded_model.eval()
loaded_model

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [4]:
#我们可以使用 model.parameters() 查看保存前和保存后的模型参数是否发生变化：
# 保存前
print("保存前：")
for param in model.parameters():
    print(param)
print("=====================================")
# 加载后
print("保存后：")
for param in loaded_model.parameters():
    print(param)

保存前：
Parameter containing:
tensor([[ 0.2545,  0.0153, -0.2169, -0.1999, -0.1549,  0.3730]],
       requires_grad=True)
Parameter containing:
tensor([-0.2678], requires_grad=True)
保存后：
Parameter containing:
tensor([[ 0.2545,  0.0153, -0.2169, -0.1999, -0.1549,  0.3730]],
       requires_grad=True)
Parameter containing:
tensor([-0.2678], requires_grad=True)


In [6]:
'''
模型参数的保存与加载
由于模型除了参数之外还存在模型结构等内容，保存整个模型的文件一般都会比只保存模型参数的文件大得多。
因此，我们在训练过程中都会选择只保存模型参数。

我们可以使用 model.state_dict() 将模型参数转为字典对象，即每层网络结构的参数分开，如下：
'''
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[ 0.2545,  0.0153, -0.2169, -0.1999, -0.1549,  0.3730]])), ('linear.bias', tensor([-0.2678]))])


In [7]:
#模型参数的保存，其实就是对上面这种字典对象的保存。我们可以使用 torch.save(model.state_dict(), FILE) 对模型参数进行保存。
FILE1 = "./model/tmpmodel1.pth"
torch.save(model.state_dict(), FILE1)
print("保存成功")

保存成功


In [10]:
'''
由于此时我们只保存了模型的参数，因此在加载模型时，我们需要提前指定模型的网络结构。
如果指定的网络结构和我们定义的模型参数不匹配，则会报错。
'''
# 指定网络结构
loaded_model = Model(n_input_features=6)
# 加载参数
dicts = torch.load(FILE1)
loaded_model.load_state_dict(dicts)
print(loaded_model)
print(loaded_model.state_dict())


'''
从结果可以看出，loaded_model.load_state_dict(torch.load(FILE)) 可以很好的加载模型的参数。

综上，如果我们保存的是整个模型，那么我们就可以直接加载该模型。
如果我们保存的仅仅是模型的参数，那么在加载模型之前我们就需要先定义模型的网络结构。
'''

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)
OrderedDict([('linear.weight', tensor([[ 0.2545,  0.0153, -0.2169, -0.1999, -0.1549,  0.3730]])), ('linear.bias', tensor([-0.2678]))])


In [11]:
'''
模型上下文的保存
为了防止在模型训练时，突然发生不可预料的错误（如服务器停电等），我们一般会定时对训练的模型进行保存。
如果发生了不可预料的情况，我们就可以从该时间点开始继续训练模型，进而避免重新开始训练模型所造成的高额成本。

当然，为了能够更好的恢复训练现场，我们除了保存该时间点下的模型参数之外，还可能需要保存一下模型训练时的变量（即模型上下文），
比如已迭代次数和当前学习率等变量。

为了方便讲解，让我们来定义一个学习率和优化器：
'''

learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [12]:
'''
我们可以将模型参数、优化器参数、迭代次数等都封装到一个字典中：
'''
checkpoint = {
    "epoch": 90,
    "model_state": model.state_dict(),
    "optim_state": optimizer.state_dict()
}
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1]}]}


In [13]:
'''
同样，我们需要使用 torch.save(checkpoint, FILE) 保存该时间点的模型上下文：
'''
FILE = "./model/tmpcheckpoint.pth"
torch.save(checkpoint, FILE)
print("保存成功")

保存成功


In [14]:
'''
接下来，让我们加载保存在本地的时间点。

由于这里保存的仅仅只是参数。因此，我们在加载本地文件之前，都需要指定网络结构和优化器类型：
'''
model = Model(n_input_features=6)
# 这里先将学习率设置为 0，观察加载模型后，是否发生变化
optimizer = optimizer = torch.optim.SGD(model.parameters(), lr=0)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [15]:
#我们还是利用 torch.load(FILE) 加载模型参数、优化器参数、迭代次数等变量：
checkpoint = torch.load(FILE)

# 加载的文件是一个字典，根据key值，将其加载到模型、优化器、迭次次数中
model.load_state_dict(checkpoint['model_state'])
optimizer.load_state_dict(checkpoint['optim_state'])
epoch = checkpoint['epoch']

model.eval()

print("优化器内参数：", optimizer.state_dict())
print("epoch:", epoch)

优化器内参数： {'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1]}]}
epoch: 90
